In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from statsmodels.stats.weightstats import *
from matplotlib import pyplot as plt
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


В этом задании нужно подготовиться к последующему процессу построения модели и оценки её качества. Это важный шаг в процессе построения и оптимизации модели, а также выбора и оценки финального решения. До того, как вы непосредственно начнете что-то обучать, важно понять, а как, вообще говоря, это оценивать? Как лучше грамотно организовать процедуру оценки, какие узкие места могут быть? Часто, адекватная оценка качества бывает не менее важной, чем непосредственно разработка признаков, обучение модели и оптимизация её параметров. 

In [3]:
train = pd.read_csv('orange_small_churn_data.train')
labels = pd.read_csv('orange_small_churn_labels.train',header=None,names=['label'])
#добавим целевую переменную и создадим общий dataframe
train['label']=labels
train.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
0,NaN,NaN,NaN,NaN,NaN,3052.0,NaN,NaN,NaN,NaN,...,vr93T2a,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
1,NaN,NaN,NaN,NaN,NaN,1813.0,7.0,NaN,NaN,NaN,...,6hQ9lNX,LM8l689qOp,NaN,ELof,xb3V,RAYp,55YFVY9,mj86,NaN,-1
2,NaN,NaN,NaN,NaN,NaN,1953.0,7.0,NaN,NaN,NaN,...,catzS2D,LM8l689qOp,NaN,NaN,FSa2,ZI9m,ib5G6X1eUxUn6,mj86,NaN,-1
3,NaN,NaN,NaN,NaN,NaN,1533.0,7.0,NaN,NaN,NaN,...,e4lqvY0,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
4,NaN,NaN,NaN,NaN,NaN,686.0,7.0,NaN,NaN,NaN,...,MAz3HNj,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


На основании полученных изображений и таблиц, предположите, какие переменные окажут наибольшее влияние (вклад) в модель?  
Из числовых переменных: Var131, Var69, Var53, (Var177, Var114, Var36)- есть корреляция этих признаков), Var139  
Из категориальных: Var218, Var225

Начнем с простого, но важного шага. Отделите небольшую выборку от существующих данных. Назовем её hold-out dataset. Эта выборка нужна для контроля качества решения: она не должна использоваться вплоть до контроля качества решения. Наличие такой выборки поможет убедиться, что в процессе моделирования не было допущено ошибок, не произошло переобучение. В качестве ответа загрузите полученный файл (или файлы, если вы работаете а данными и метками как с 2мя файлами)

In [13]:
from sklearn.model_selection import train_test_split

train_train, train_test = train_test_split(train, 
                                                train_size=0.9, 
                                                random_state=42)

In [14]:
train_train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
20038,NaN,NaN,NaN,NaN,NaN,119.0,0.0,NaN,NaN,NaN,...,fXVEsaq,NaN,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN,1
23937,NaN,NaN,NaN,NaN,256535.0,NaN,NaN,NaN,NaN,5661.0,...,BSis5Dl,LM8l689qOp,NaN,NaN,fKCe,02N6s8f,F2FyR07IdsN7I,NaN,NaN,-1
6046,NaN,NaN,NaN,NaN,NaN,1043.0,7.0,NaN,NaN,NaN,...,NKvz4Af,LM8l689qOp,NaN,kG3k,WqMG,RAYp,55YFVY9,NaN,NaN,-1
23187,NaN,NaN,NaN,NaN,NaN,371.0,0.0,NaN,NaN,NaN,...,EXUR6zD,LM8l689qOp,NaN,NaN,453m,RAYp,0pzBWGkV3fbsGZN52DH,NaN,NaN,-1
25421,NaN,NaN,NaN,NaN,NaN,644.0,7.0,NaN,NaN,NaN,...,5gSwS9i,LM8l689qOp,NaN,NaN,uWr3,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,NaN,NaN,NaN,NaN,NaN,1064.0,7.0,NaN,NaN,NaN,...,PgO2Oqu,LM8l689qOp,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
11284,NaN,NaN,NaN,NaN,NaN,1246.0,7.0,NaN,NaN,NaN,...,K2SqEo9,LM8l689qOp,NaN,ELof,szEZ,6fzt,4rd_,NaN,NaN,-1
38158,NaN,NaN,NaN,NaN,NaN,777.0,7.0,NaN,NaN,NaN,...,W41oG9v,M_8D,NaN,kG3k,Qu4f,RAYp,F2FyR07IdsN7I,mj86,NaN,-1
860,NaN,NaN,NaN,NaN,NaN,504.0,NaN,NaN,NaN,NaN,...,05j67Xw,LM8l689qOp,NaN,ELof,rgKb,RAYp,R4y5gQQWY8OodqDV,am7c,NaN,-1


In [15]:
train_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230,label
32823,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,76DJixu,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
16298,NaN,NaN,NaN,NaN,NaN,2632.0,7.0,NaN,NaN,NaN,...,x3a7KuW,LM8l689qOp,NaN,ELof,Qcbd,nIGXDli,ib5G6X1eUxUn6,mj86,NaN,-1
28505,NaN,0.0,192.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,xKQ2fT4,NaN,4n2X,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
6689,NaN,NaN,NaN,NaN,NaN,770.0,7.0,NaN,NaN,NaN,...,CnZoIp0,jySVZNlOJy,NaN,NaN,WqMG,RAYp,F2FyR07IdsN7I,am7c,NaN,-1
26893,NaN,NaN,NaN,NaN,NaN,980.0,0.0,NaN,NaN,NaN,...,JyKROSq,LM8l689qOp,NaN,NaN,FSa2,02N6s8f,xwM2aC7IdeMC0,NaN,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19536,NaN,NaN,NaN,NaN,NaN,1897.0,7.0,NaN,NaN,NaN,...,dxUynIC,LM8l689qOp,NaN,NaN,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN,-1
13332,NaN,NaN,NaN,NaN,NaN,931.0,7.0,NaN,NaN,NaN,...,eEfa_vf,LM8l689qOp,NaN,NaN,FSa2,RAYp,F2FyR07IdsN7I,NaN,NaN,-1
18523,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,...,RJNqhZX,jySVZNlOJy,NaN,xG3x,7P5s,RAYp,F2FyR07IdsN7I,mj86,NaN,1
14835,NaN,NaN,NaN,NaN,NaN,784.0,7.0,NaN,NaN,NaN,...,rH0P_83,LM8l689qOp,NaN,NaN,Qu4f,RAYp,F2FyR07IdsN7I,NaN,NaN,-1


In [16]:
train_test.to_csv('hold-out_dataset.csv') 

In [17]:
train_train.to_csv('train.csv') 